# Φάση 2 – Σημασιολογική ανάκτηση χρησιμοποιώντας Transformers και FAISS

#### Ομάδα: Γιαννακοπούλου Μαρία-Βασιλική 3220026, Μανιάτης Ιωάννης 3220114

In [1]:
!pip install pandas tqdm requests
!pip install transformers sentence-transformers torch
!pip install faiss-cpu 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


# Imports

In [2]:
import pandas as pd
from pathlib import Path
import html
import re
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np
import os

# Ρυθμίσεις paths (σύμφωνα με την παρακάτω δομή)
```text
├── Phase2.ipynb
└── data
    ├── documents.csv
    ├── qrels.csv
    ├── qrels.txt
    ├── queries.csv
    └── queries.txt

In [3]:
CWD = Path.cwd()
DATA_DIR = CWD / "data" 
DOCUMENTS_CSV = DATA_DIR / "documents.csv"

# 2.1 Preprocessing
- Δεν αφαιρούμε stop-words καθώς τα transformers χρειάζονται τα συμφραζόμενα για να "καταλάβουν" τη σχέση μεταξύ λέξεων
- Το tokenization γίνεται από το ίδιο το μοντέλο στη συνέχεια

In [4]:
# Συνάρτηση Καθαρισμού Κειμένου
def clean_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s)
    s = html.unescape(s)
    s = re.sub(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]+", " ", s)
    s = s.replace('\r', ' ').replace('\n', ' ')
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Διάβασμα CSV
df_docs = pd.read_csv(DOCUMENTS_CSV).fillna("")

print(f"Βρέθηκαν {len(df_docs)} έγγραφα.")
print(f"Στήλες αρχείου: {list(df_docs.columns)}")

# Καθαρίζουμε τη στήλη 'Text' και τη βάζουμε στο 'full_text'
if 'Text' in df_docs.columns:
    df_docs['full_text'] = df_docs['Text'].apply(clean_text)
else:
    raise ValueError("Δεν βρέθηκε στήλη 'Text'")

print("\nΔείγμα:")
print("-" * 50)
print(df_docs.head())

Βρέθηκαν 18316 έγγραφα.
Στήλες αρχείου: ['ID', 'Text']

Δείγμα:
--------------------------------------------------
       ID                                               Text  \
0  193157  Support towards the Europe PMC initiative-Cont...   
1  193158  Support to the Vice-Presidents of the ERC Scie...   
2  193159  Implementation of activities described in the ...   
3  193160  Monitoring Atmospheric Composition and Climate...   
4  193161  Pre-Operational Marine Service Continuity in T...   

                                           full_text  
0  Support towards the Europe PMC initiative-Cont...  
1  Support to the Vice-Presidents of the ERC Scie...  
2  Implementation of activities described in the ...  
3  Monitoring Atmospheric Composition and Climate...  
4  Pre-Operational Marine Service Continuity in T...  


# 2.2 Δημιουργία embeddings
Θα χρησιμοποιήσουμε το μοντέλο `all-MiniLM-L6-v2` καθώς είναι γρήγορο, μικρό και αποδοτικό για `semantic search` όπως αναφέρεται και στις αντίστοιχες διαφάνειες του μαθήματος

In [16]:
# Ορίζουμε το μοντέλο (all-MiniLM-L6-v2)
model_name = 'all-MiniLM-L6-v2'
#model = SentenceTransformer(model_name)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Μετατροπή στήλης full_text σε λίστα
sentences = df_docs['full_text'].tolist()

print(f"Δημιουργία embeddings για {len(sentences)} κείμενα...")

# Παραγωγή των embeddings
doc_embeddings = model.encode(sentences, convert_to_numpy=True, show_progress_bar=True)


# Διαστάσεις
print(f"\nShape of embeddings: {doc_embeddings.shape}")

KeyboardInterrupt: 